In [ ]:
!pip install "huggingface_hub[cli]" datasets transformers trl torch wandb ipykernel ipywidget iprogress

In [2]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: embereagle to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
from datasets import load_dataset

# https://huggingface.co/datasets/HuggingFaceTB/smoltalk
dataset = load_dataset("HuggingFaceTB/smoltalk", 'all')

/home/ubuntu/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['messages', 'source'],
        num_rows: 1043917
    })
    test: Dataset({
        features: ['messages', 'source'],
        num_rows: 54948
    })
})

In [7]:
dataset['train'][0]

{'messages': [{'content': 'The function \\( g(x) \\) satisfies the functional equation\n\\[ g(x + y) = g(x) + g(y) \\]\nfor all real numbers \\( x \\) and \\( y \\), and it is given that \\( g(3) = 4 \\). Find \\( g(10) \\).',
   'role': 'user'},
  {'content': 'Given the functional equation and the specific value \\( g(3) = 4 \\), we can find \\( g(1) \\) by using the equation multiple times:\n\\[\ng(3) = g(2) + g(1)\n\\]\n\\[\ng(2) = g(1) + g(1) = 2g(1)\n\\]\nThus,\n\\[\n4 = 2g(1) + g(1) = 3g(1)\n\\]\n\\[\ng(1) = \\frac{4}{3}\n\\]\nNow we can find \\( g(10) \\) using \\( g(1) \\):\n\\[\ng(10) = 10g(1) = 10 \\times \\frac{4}{3} = \\frac{40}{3}\n\\]\nHence, the value of \\( g(10) \\) is \\(\\boxed{\\frac{40}{3}}\\).',
   'role': 'assistant'}],
 'source': 'numina-cot-100k'}

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# https://huggingface.co/microsoft/DialoGPT-medium
# model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
# tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")

# https://huggingface.co/Qwen/Qwen2.5-0.5B
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B")
# tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B", use_fast=True)

# https://huggingface.co/distilbert/distilgpt2
model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2")
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2", use_fast=True)

In [9]:
import multiprocessing
num_proc = multiprocessing.cpu_count()

def chatml_tokenize(batch):
    texts = []
    for messages in batch["messages"]:
        chat = ""
        for msg in messages:
            if msg["role"] == "user":
                chat += "<|user|> " + msg["content"].strip() + " " + tokenizer.eos_token + " "
            elif msg["role"] == "assistant":
                chat += "<|assistant|> " + msg["content"].strip() + " " + tokenizer.eos_token + " "
        texts.append(chat.strip())
    return tokenizer(texts, padding=False, truncation=False)

tokenized_train = dataset["train"].map(
    chatml_tokenize, batched=True, batch_size=1000, num_proc=num_proc, remove_columns=["messages"]
)
tokenized_test = dataset["test"].map(
    chatml_tokenize, batched=True, batch_size=1000, num_proc=num_proc, remove_columns=["messages"]
)

In [ ]:
# Should show ChatML-formatted text
print("Sample training example:")
print(tokenizer.decode(tokenized_train[0]["input_ids"])) 
# Expected: "<|user|> ... <|assistant|> ..."

In [10]:
import torch

device = "mps" if torch.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)
print(device)

cpu


In [ ]:
# Add special tokens to tokenizer
special_tokens = ["<|user|>", "<|assistant|>"]
tokenizer.add_special_tokens({"additional_special_tokens": special_tokens})
model.resize_token_embeddings(len(tokenizer))

In [ ]:
# Evaluate WITHOUT ChatML formatting
def base_model_eval(question):
    encoded = tokenizer(question, return_tensors="pt").to(device)
    generated = model.generate(**encoded, max_new_tokens=20)
    return tokenizer.decode(generated[0], skip_special_tokens=True)

print("BEFORE TRAINING (Raw model):")
print(base_model_eval("The capital of France is"))
print(base_model_eval("What is the capital of France?"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The capital of France is ÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂ
What is the capital of France?






















In [ ]:
!pip install tf-keras

In [ ]:
import random

# sample random indices from the test set
random_indices = random.sample(range(len(tokenized_test)), 400)

# create a new Dataset with only those
sampled_eval_dataset = tokenized_test.select(random_indices)

In [13]:
# The trainer crashes with wandb active, even in offline mode .. login worked (above). The quick fix is to deactivate wandb

import os
# os.environ["WANDB_MODE"] = "offline"
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from transformers import DataCollatorForLanguageModeling
from trl import SFTConfig, SFTTrainer

# Enable gradient checkpointing to reduce memory
# model.gradient_checkpointing_enable()
# model.config.use_cache = False

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as padding if not set

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = SFTConfig(
    output_dir="./trainer_output",
    per_device_train_batch_size=4,
    max_steps=50,
    learning_rate=1e-5,
    bf16=True,
    # fp16=True, # backup option
    logging_steps=10,
    save_total_limit=2,  # Keep last 2 checkpoints
    save_strategy="steps",
    save_steps=50,  # Save every 50 steps
    eval_strategy="steps",
    eval_steps=50,
    dataloader_num_workers=4,
    optim="adamw_torch_fused",
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    num_train_epochs=1,
    run_name="infomaniak_gpu"
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=sampled_eval_dataset,
    data_collator=data_collator,
)

trainer.train()


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/home/ubuntu/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss


In [ ]:
# Evaluate WITH ChatML formatting
def chatml_eval(question):
    formatted_prompt = f"<|user|> {question} <|assistant|>"
    encoded = tokenizer(formatted_prompt, return_tensors="pt").to(device)
    generated = model.generate(**encoded, max_new_tokens=100)
    return tokenizer.decode(generated[0], skip_special_tokens=True)

print("\nAFTER TRAINING (ChatML-formatted):")
print(chatml_eval("The capital of France is"))
print('...')
print(chatml_eval("What is the capital of France?"))

In [ ]:
assert model.get_output_embeddings().weight.data_ptr() == model.get_input_embeddings().weight.data_ptr()

In [ ]:
# Continue training, use last checkpoint
trainer.args.max_steps += 1000
trainer.train(resume_from_checkpoint=True)

In [ ]:
# Evaluate WITH ChatML formatting
def chatml_eval(question):
    formatted_prompt = f"<|user|> {question} <|assistant|>"
    encoded = tokenizer(formatted_prompt, return_tensors="pt").to(device)
    generated = model.generate(**encoded, max_new_tokens=100)
    return tokenizer.decode(generated[0], skip_special_tokens=True)

print("\nAFTER TRAINING (ChatML-formatted):")
print(chatml_eval("The capital of France is"))
print('...')
print(chatml_eval("What is the capital of France?"))